In [15]:
import sys
#!{sys.executable} -m pip install wordcloud
#!{sys.executable} -m pip install pysentimiento
!{sys.executable} -m pip install --upgrade pandas

     -------------------------------------- 10.6/10.6 MB 456.8 kB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.1
    Uninstalling pandas-1.4.1:
      Successfully uninstalled pandas-1.4.1


### Import Libraries

In [3]:

# utilities
import re
import numpy as np
import pandas as pd
import pickle 
# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# nltk
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
# To read faster the large data 
from dask import dataframe as df1


### Read Data

In [12]:
# Importing the dataset
elongate_tweets_file = 'raw_data/elongate_tweets.csv'
regular_tweets_file = 'raw_data/elonmusktwitter_tweets.csv'
chunk = pd.read_csv(elongate_tweets_file, sep=',', error_bad_lines=False, index_col=False, dtype='unicode', chunksize=1000)
elongate_df = pd.concat(chunk)

chunk = pd.read_csv(regular_tweets_file, sep=',', error_bad_lines=False, index_col=False, dtype='unicode', chunksize=1000)
regular_df = pd.concat(chunk)

elongate_df.sample(5)


,date,id,content,lang,replyCount,retweetCount,likeCount,inReplyToTweetId,inReplyToUser,mentionedUsers
419795,2022-05-03 05:37:25+00:00,1521363275576971264,@Kristennetten @elonmusk @etnow His momma look...,en,0,0,0,1.5212955383766917e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u..."
69626,2022-05-04 16:24:47+00:00,1521888579416301570,@elonmusk Advice taken. Thank you,en,0,0,0,1.5206500368659497e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u..."
35602,2022-05-04 19:48:21+00:00,1521939809643556864,@Caper_Creations @elonmusk It was normal when ...,en,1,0,1,1.521515653068464e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u..."
102912,2022-05-04 13:42:50+00:00,1521847824035557376,@bhensel2009 @HotSauc95211743 @BorislavKostad1...,en,1,0,1,1.5218404826613268e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u..."
28513,2022-05-04 20:37:47+00:00,1521952251752984576,Biden should stop the free c19 home test to th...,en,0,0,0,NaN,NaN,"[{'_type': 'snscrape.modules.twitter.User', 'u..."


In [8]:
print(elongate_df.size)
elongate_filter = (elongate_df['lang'] == 'en')

elongate_df = elongate_df[elongate_filter]
print(elongate_df.size)
elongate_df.head(5)

3597810
3597810


,date,id,content,lang,replyCount,retweetCount,likeCount,inReplyToTweetId,inReplyToUser,mentionedUsers
0,2022-05-04 23:59:59+00:00,1522003135794929665,@elonmusk @bevedoni It would make us happy if ...,en,0,0,0,1.5219985677574267e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u..."
1,2022-05-04 23:59:59+00:00,1522003134612127746,@Crypto_Highland @RatsoLicksy @CliffToken @elo...,en,1,0,2,1.522002058949415e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u..."
2,2022-05-04 23:59:59+00:00,1522003134557663235,@elonmusk @bevedoni Word !,en,0,0,0,1.5219985677574267e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u..."
3,2022-05-04 23:59:59+00:00,1522003134238715910,@teslaownersSV @elonmusk He was joking and her...,en,0,0,0,1.5217191082528236e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u..."
4,2022-05-04 23:59:58+00:00,1522003132162658304,@elonmusk @bevedoni It’s very true!,en,0,0,0,1.5219985677574267e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u..."


In [10]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="en")



In [1]:
import pandas as pd
pd.__version__
#%pip install pandas==1.4.1

'1.4.2'

In [4]:
f3 = open('raw_data/elonmusktwitter_tweets_df.obj', 'rb')
df3 = pickle.load(f3)
df3.size

22132290

In [28]:
def analyze(content):
  res = analyzer.predict(content)
  return res.output

df_sample = df2.sample(n=1000)
df_sentiment = [analyze(x) for x in df_sample['content']]
df_sentiment[:10]

['NEG', 'NEG', 'NEG', 'NEU', 'NEG', 'NEG', 'NEG', 'NEG', 'POS', 'NEU']

In [29]:
df2.size

17753355